In [8]:
import torch
import numpy as np
import pandas as pd
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torchvision.models import mobilenet_v2
import torch.nn as nn
import os
import cv2
import random
import time
import pickle
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

In [9]:
img_rows, img_cols = 224, 224
batch_size = 16
Epoch = 5
random_state = 51
torch.manual_seed(random_state)

In [24]:
data_transform = transforms.Compose([
        transforms.RandomRotation(10, expand=True),
        transforms.ColorJitter(brightness=[0,1], contrast=[0,1], saturation=[0,1], hue=[-0.5,0.5]),
        transforms.RandomHorizontalFlip(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.2, 0.2, 0.2])
    ])

In [25]:
PATH = 'C:/Users/Vedant/Desktop/ML_Assignment_201701076/Driver Distraction Problem/imgs/train'

In [26]:
dataset = datasets.ImageFolder(root=PATH,transform=data_transform)

In [27]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
test_dataset,valid_dataset = random_split(val_dataset, [int(0.5*val_size), val_size - int(0.5*val_size)])

In [28]:
tag2class = {'0': 'normal driving','1': 'texting - right','2': 'talking on the phone - right','3': 'texting - left','4': 'talking on the phone - left','5': 'operating the radio','6': 'drinking','7': 'reaching behind','8': 'hair and makeup','9': 'talking to passenger'}

In [29]:
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
validation_dataloader = DataLoader(
            valid_dataset, # The validation samples.
            sampler = SequentialSampler(valid_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [16]:
model = mobilenet_v2(pretrained=True, progress=True)

In [17]:
model.classifier = nn.Sequential(nn.Dropout(p=0.3),nn.Linear(1280,10),nn.Softmax(dim = 0))

In [18]:
model.cuda()

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [19]:
import torch.optim as optim
import torch.nn.functional as F

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)

In [20]:
import numpy as np
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [21]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
EPOCHS = 25

training_stats = []
total_eval_accuracy = 0
total_train_accuracy = 0
best_avg_val_accuracy = 0

for epoch in range(0,EPOCHS):  # loop over the dataset multiple times
    
    t0 = time.time()
    running_loss = 0.0
    total_train_accuracy = 0
    
    for i,batch in enumerate(train_dataloader):
        
        if i % 100 == 0 and not i == 0:
            elapsed = format_time(time.time() - t0)
            print('Epoch {}  Batch {:>5,}  of  {:>5,}.  Elapsed: {:}. Loss {}'.format(epoch,i, len(train_dataloader), elapsed,
                                                                                       avg_train_loss))
        
        inputs, labels = batch
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.cuda())
        
        loss = criterion(outputs,labels.type(torch.LongTensor).cuda())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
        avg_train_loss = running_loss / len(train_dataloader)
        outputs = outputs.detach().cpu().numpy()
        label_ids = labels.detach().to('cpu').numpy()

        total_train_accuracy += flat_accuracy(outputs, label_ids)
        

    avg_train_accuracy = total_train_accuracy / len(train_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_train_accuracy))
    
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    print("")
    print("Running Validation...")

    model.eval()
 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        
        inputs, labels = batch
        with torch.no_grad():        
            outputs = model(inputs.cuda())
            
        # Accumulate the validation loss.
        loss = criterion(outputs,labels.type(torch.LongTensor).cuda())
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        outputs = outputs.detach().cpu().numpy()
        label_ids = labels.detach().to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(outputs, label_ids)
        

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    
    training_stats.append(
        {
            'epoch': epoch + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
        }
    )
    
    if best_avg_val_accuracy < avg_val_accuracy:
        best_avg_val_accuracy = avg_val_accuracy
        if epoch > 10:
            torch.save(model.state_dict(), 'working/checkpoint_'+str(epoch)+'.pth')
print('Finished Training')

In [22]:
PATH = 'C:/Users/Vedant/Desktop/ML_Assignment_201701076/Driver Distraction Problem/MNv2 - Preprocessing/checkpoint_24.pth'
model.load_state_dict(torch.load(PATH))
model.eval()

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
total_test_accuracy = 0
total_test_loss = 0

y_true = []
y_pred = []

for i, batch in enumerate(test_dataloader):

    inputs, labels = batch
    with torch.no_grad():        
        outputs = model(inputs.cuda())

    # Accumulate the validation loss.
    loss = criterion(outputs,labels.type(torch.LongTensor).cuda())
    total_test_loss += loss.item()

    # Move logits and labels to CPU
    outputs = outputs.detach().cpu().numpy()
    label_ids = labels.detach().to('cpu').numpy()
    
    y_pred.append(outputs)
    y_true.append(label_ids)
    

    total_test_accuracy += flat_accuracy(outputs, label_ids)

avg_test_accuracy = total_test_accuracy / len(test_dataloader)
print("  Accuracy: {0:.2f}".format(avg_test_accuracy))

# Calculate the average loss over all of the batches.
avg_test_loss = total_test_loss / len(test_dataloader)
print("  Test Loss: {0:.2f}".format(avg_test_loss))

In [ ]:
y_true = np.concatenate(y_true,axis=0)

In [ ]:
y_pred = np.concatenate(y_pred,axis=0)
y_pred = np.argmax(y_pred, axis=1).flatten()

In [ ]:
from sklearn.metrics import classification_report

target_names = ['normal driving','texting - right','talking on the phone - right','texting - left','talking on the phone - left','operating the radio','drinking','reaching behind','hair and makeup','talking to passenger']
print(classification_report(y_true, y_pred, target_names=target_names))